In [1]:
%load_ext autoreload
%autoreload 2

In [1]:
import matplotlib.pyplot as plt
from gps_accuracy.gps_accuracy import GpxResult
from gps_accuracy.gps_accuracy import GpxEvaluator
import pandas as pd
import itertools
from dataclasses import dataclass
from enum import Enum
from pathlib import Path
from typing import List

class InputType(Enum):
    Touch = 1
    TUI = 2
    
class Metaphor(Enum):
    Gesture = 1
    Joystick = 2
    Car =  3
    
@dataclass
class ReferenceTrack:
    track_id: int
    file: Path
    
    def __init__(self, file_path: Path):
        self.track_id = int(file_path.stem)
        self.file = file_path

@dataclass
class RecordedTrack:
    track_id: int
    user_id: int
    input_type: InputType
    metaphor: Metaphor
    file: Path
    result: GpxResult
    
    def __init__(self, file_path: Path):
        file_name = file_path.stem
        parts = file_name.split("_")
        self.user_id: int = int(parts[0])
        self.track_id: int = int(parts[1])
        self.input_type: InputType = InputType[parts[2]]
        self.metaphor: Metaphor = Metaphor[parts[3]]
        self.file: Path = file_path
    
    def evaluate(self, reference_track: ReferenceTrack):
        evaluator = GpxEvaluator(reference_track.file, self.file)
        self.result = evaluator.evaluate()
              
@dataclass
class TrackRepository:
    reference_tracks: dict
    recorded_tracks: List[RecordedTrack]
    
    def __init__(self):
        reference_track_list = [ReferenceTrack(track_file) for track_file in Path("reference_tracks").iterdir() if track_file.is_file()]
        self.reference_tracks = {track.track_id: track for track in reference_track_list}
        self.recorded_tracks = [RecordedTrack(track_file) for track_file in Path("recorded_tracks").iterdir() if track_file.is_file()]
        self._evaluate() 
    
    def _evaluate(self):
        for track in self.recorded_tracks:
            reference_track = self.reference_tracks[track.track_id]
            track.evaluate(reference_track)
        
    def get_by_track(self, track_id: int) -> List[RecordedTrack]:
        return [track for track in self.recorded_tracks if track.track_id == track_id]

    def get_by_user(self, user_id: int) -> List[RecordedTrack]:
        return [track for track in self.recorded_tracks if track.user_id == user_id]
    
    def get_by_input_type(self, input_type: InputType) -> List[RecordedTrack]:
        return [track for track in self.recorded_tracks if track.input_type == input_type]
    
    def get_by_metaphor(self, metaphor: Metaphor) -> List[RecordedTrack]:
        return [track for track in self.recorded_tracks if track.metaphor == metaphor]
    
    def get_all(self) -> List[RecordedTrack]:
        return self.recorded_tracks

class InputFilter(Enum):
    InputAll = 1
    InputCategorized = 2
    
class ResultParam(Enum):
    Time = 1
    MeanError = 2
    MedianError = 3
    PercentileError = 4
    Distance = 5
    DeltaDistance = 6
    ZoomMin = 7
    ZoomMax = 8
    ZoomMean = 9
    ZoomChange = 10

class ResultPlotter:
    def __init__(self, user_ids: List[int] = None):
        self.repo = TrackRepository()
        tracks = list(itertools.chain(*[self.repo.get_by_user(user_id) for user_id in user_ids])) if user_ids else self.repo.get_all()
        data = {
            'UserId': [track.user_id for track in tracks],
            'Track':  [track.track_id for track in tracks],
            'InputAll': [f"{track.input_type.name}_{track.metaphor.name}" for track in tracks],
            'InputCategorized': [track.input_type.name for track in tracks],
            ResultParam.Time.name: [track.result.time for track in tracks],
            ResultParam.MeanError.name: [track.result.error_mean for track in tracks],
            ResultParam.MedianError.name: [track.result.error_median for track in tracks],
            ResultParam.PercentileError.name: [track.result.error_percentile for track in tracks],
            ResultParam.Distance.name: [track.result.distance for track in tracks],
            ResultParam.DeltaDistance.name: [track.result.delta_distance for track in tracks],
            ResultParam.ZoomMin.name: [track.result.zoom_min for track in tracks],
            ResultParam.ZoomMax.name: [track.result.zoom_max for track in tracks],
            ResultParam.ZoomMean.name: [track.result.zoom_mean for track in tracks],
            ResultParam.ZoomChange.name: [track.result.zoom_change for track in tracks],
        }
        self.data_frame = pd.DataFrame(data)
        self.data_frame.style.format(precision=2, decimal=".")
    
    def summary(self):
        return self.data_frame.style.format(precision=2, )
    
    def print_result(self, result_param: ResultParam, input_filter: InputFilter, aggfunc: str, min:float = None, max:float = None, plot = False, color=False):
        table = self.data_frame.pivot_table(index=input_filter.name, columns="Track", values=result_param.name, aggfunc=[aggfunc], sort=False)
        if plot:
            plt.figure()
            table.plot.bar()
        style = table.style
        if color:
            style = style.background_gradient(axis=0, cmap='Reds', vmin=min, vmax=max)
        return style.format(precision=2)

    

In [3]:
plotter = ResultPlotter([11, 12])
plotter.summary()

,UserId,Track,InputAll,InputCategorized,Time,MeanError,MedianError,PercentileError,Distance,DeltaDistance,ZoomMin,ZoomMax,ZoomMean,ZoomChange
0,11,1,Touch_Gesture,Touch,125.00,0.77,0.61,2.01,2110.81,53.14,19.03,19.48,19.08,0.45
1,11,1,Touch_Joystick,Touch,64.00,0.79,0.38,3.68,2086.03,28.36,19.77,19.77,19.77,0.00
2,11,1,TUI_Car,TUI,81.00,1.50,1.03,4.80,2065.49,7.82,17.66,18.19,18.09,0.53
3,11,1,TUI_Joystick,TUI,96.00,0.71,0.52,2.25,2057.64,-0.02,19.20,19.50,19.43,0.30
4,11,2,Touch_Gesture,Touch,148.00,1.25,0.93,3.51,2558.20,143.05,19.39,20.19,19.74,1.77
5,11,2,Touch_Joystick,Touch,68.00,0.70,0.62,1.34,2418.00,2.85,19.97,19.97,19.97,0.00
6,11,2,TUI_Car,TUI,105.00,1.09,0.81,3.01,2423.14,7.99,18.07,18.40,18.38,0.33
7,11,2,TUI_Joystick,TUI,81.00,1.00,0.89,2.56,2414.35,-0.80,19.07,19.61,19.59,0.54
8,11,3,Touch_Gesture,Touch,54.00,1.30,0.93,3.79,988.42,55.32,19.42,20.18,19.66,1.20
9,11,3,Touch_Joystick,Touch,47.00,0.72,0.52,1.98,924.26,-8.84,20.13,20.13,20.13,0.00


In [10]:
plotter.print_result(ResultParam.MeanError, InputFilter.InputAll, aggfunc='mean', color=True, plot=False)

In [11]:
plotter.print_result(ResultParam.MedianError, InputFilter.InputAll, aggfunc='mean', color=True, plot=False)

In [12]:
plotter.print_result(ResultParam.DeltaDistance, InputFilter.InputAll, aggfunc='mean', color=True, plot=False)

In [13]:
plotter.print_result(ResultParam.Time, InputFilter.InputAll, aggfunc='mean', color=True, plot=False)